In [ ]:
import sys
import random
import logging
import config
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = np.load(open("simulation_outcome.train111_0.npy", 'rb'))[()]

In [ ]:
'''
Check shapes of the data.
'''
print("Keys: ",', '.join([key for key in data.keys()]), "\n")
for key in data.keys():
    if type(data[key]) is np.ndarray: 
        print("Shape %-11s %s" % (key, str(data[key].shape)))

In [ ]:
'''
Load data into variables for nicer access.
'''
z = data['z']
c = data['centroids_z']
s = data['s']
mu = data['mu'][:,:]
y = data['y'][:,:]
x = data['x'][:,:]
t = data['t'][:]
n_treatments = c.shape[0]
n_samples = z.shape[0]

for i in range(3):
    print("Nr t=%d: %d" % (i, sum(t==i)))
    

In [ ]:
'''
Dot product to centroid in topic space. 
Used for treatment assignment and outcome calculation.
'''
dist = np.zeros([n_treatments, n_samples])
for i in range(n_treatments):
    dist[i] = [np.dot(c[i],j) for j in z]

In [ ]:
plt.rcParams['figure.figsize'] = (20.0, 7.0)

for i in range(1,n_treatments):
    if data["treatment_types"][i-1] > 0:
        print("Treatment %d" % i)
        print("Min treatment strength: "+str(min(s[:,i])))
        print("Max treatment strength: "+str(max(s[:,i])))

        plt.subplot(131)
        plt.hist(s[:,i], bins=30, alpha=0.5, label='s')
        plt.xlabel('Treatment strength')
        plt.ylabel('Count')

        plt.subplot(132)
        plt.plot(dist[i], s[:,i], 'o', alpha=0.5)
        plt.xlabel('Dot-product to centroid')
        plt.ylabel('Treatment strength for t=%d' % i)

        plt.subplot(133)
        plt.plot(mu[:,i], s[:,i], 'o', alpha=0.5)
        plt.xlabel('True outcome')
        plt.ylabel('Treatment strength for t=%d' % i)

        
        plt.show()

for i in range(n_treatments):
    print("Treatment %d" % i)
    plt.subplot(121)
    plt.hist(mu[:,i], bins=np.arange(-3,10,0.5), alpha=0.3, label='mu'+str(i)+" (True)")
    plt.hist(y[:,i], bins=np.arange(-3,10,0.5), alpha=0.3, label='y'+str(i)+" (Noisy)")
    plt.xlabel('Outcome')
    plt.ylabel('Count')
    plt.legend()

    plt.subplot(122)
    plt.plot(dist[i], mu[:,i],'o', alpha=0.05, label='mu'+str(i)+" (True)", marker='.')
    plt.plot(dist[i], y[:,i],'o', alpha=0.05, label='y'+str(i)+" (Noisy)", marker='.')
    plt.xlabel('Dot-product to centroid')
    plt.ylabel('Outcome')
    plt.legend()

    plt.show()

In [ ]:
mu_f = mu[np.arange(n_samples), t.astype(int)]
plt.figure()
plt.title("Comparison (true) treatment outcome factual vs counterfactual")
plt.hist(mu_f, alpha=0.3, label="Factual")
for i in range(1, n_treatments):
    mu_cf = mu[np.arange(n_samples), np.mod(t+i,n_treatments).astype(int)]
    plt.hist(mu_cf, alpha=0.3, label="Counterfactual option"+str(i))
plt.xlabel('Outcome')
plt.ylabel('Count')
plt.legend()
plt.show()
print("Since treatment decision is based on knowledged, outcome is generally higher for the factual group.")

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def plot_DR(data, centroids, space_name, algo="pca"):
    x = data
    c_x = centroids 
    n_treatments = len(centroids)
    
    print("#centroids = %d" % len(c_x))
    print("dimension of centroids = %d" % len(c_x[0]))
    if algo is "pca":
        pca = PCA(n_components=2)
        pca = pca.fit(x)
        x_r = pca.transform(x)
        centroids_r = pca.transform(c_x)

    elif algo is "tsne":
        tsne = TSNE(n_components=2)
        tsne = tsne.fit_transform( np.concatenate((data, centroids)) )
        x_r = tsne[:-n_treatments,:]
        centroids_r = tsne[-n_treatments:,:]
    
    plt.figure()

    cmap_data = plt.cm.get_cmap('Paired')
    colors = [cmap_data(i*2+1) for i in range(n_treatments)]
    
    for color, i, target_name in zip(colors, np.arange(n_treatments), ["t"+str(i) for i in range(n_treatments)]):
        plt.scatter(x_r[t == i,0], x_r[t == i,1], color=color, alpha=0.8, lw=2, label=target_name, marker='.')
    
    for color, i, target_name in zip(colors, np.arange(n_treatments), ["centroid"+str(i) for i in range(n_treatments)]):
        centroid_r = centroids_r[i]
        plt.scatter(centroid_r[0], centroid_r[1], color=color, lw=1, label=target_name, marker='D', edgecolors='b')
    
    plt.legend(loc='best', shadow=False, scatterpoints=1)
    plt.title(algo + ' of topics in ' + space_name)
    plt.show()
    
plot_DR(x, data['centroids_x'], "word-space", algo="pca")
plot_DR(z, c, "topic-space", algo="pca")
plot_DR(z, c, "topic-space", algo="tsne")  # This takes a while